In [1]:
import pandas as pd
from yelpapi import YelpAPI

# 1. Process columns


In [4]:
df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/stockholm_housing_df_RAW.csv")

df['fee'] = df['fee'].str.rstrip(' kr/mån')

#Drop 'location' column, as it is redundant
df = df.drop(columns=["location"])

#Split size of apartment and number of rooms into separate columns
df[['size', 'rooms']] = df['size:rooms'].str.split("  ", 1, expand=True)
df = df.drop(columns=["size:rooms"])
#Clean up the columns size and room, removing m2 and "rum"
df['size'] = df['size'].str.rstrip('m²')
df["rooms"] = df["rooms"].str.rstrip(' rum')

#Split adress and floor of apt into separate columns
df[['adress', 'floor']] = df['adress'].str.split(",", 1, expand=True)

#Clean up sold_date, sale_price, value_dev, ppsqm
df["sold_date"] = df["sold_date"].str.lstrip('såld')
df["sale_price"] = df["sale_price"].str.lstrip('slutpris ')
df["sale_price"] = df["sale_price"].str.rstrip(' kr')
df["ppsqm"] = df["ppsqm"].str.rstrip(" kr/m²")
df["value_dev"] = df["value_dev"].str.rstrip(" %")
df["value_dev"] = df["value_dev"].str.lstrip("+")

# Introduce dummy variables in the following way: Balkong = 1, Hiss = 2, Balkong&Hiss = 3, Uteplats = 4
df["features"] = df.features.map( {'balkong':1 , 'hiss':2, 'balkong&hiss':3, 'uteplats': 4, 'NaN':5} )

#Remove everything except numerical values from floor
df['floor'] = df['floor'].str.extract('(\d+)', expand=False)

#Bug where value_dev ends up in wrong columns, due to old posting
bad_index = df[df["ppsqm"].str.find("%") != -1].index
df = df.drop(bad_index, axis=0)

column_list = ["ppsqm", "size", "rooms", "sale_price","fee"]

def fix_numeric(column_list, df):
    new_df = df.copy()
    for i in column_list: 
        new_df[i] = new_df[i].str.replace(" ", "")
        new_df[i] = pd.to_numeric(new_df[i] , errors='coerce')
        new_df = new_df.dropna(subset=[i])
        new_df[i] = new_df[i].astype('int')
    return new_df

cleaned_housing_df = fix_numeric(column_list,df.copy() )
#Reset index after removing rows
cleaned_housing_df = cleaned_housing_df.reset_index(drop=True)
#cleaned_housing_df.to_csv("stockholm_housing_df_CLEANED.csv",index=False)


In [5]:
cleaned_housing_df

,adress,fee,features,sale_price,sold_date,value_dev,ppsqm,district,size,rooms,floor
0,frejgatan 50,2391,1.0,6450000,15 januari 2022,NaN,153571,vasastan,42,2,None
1,rådmansgatan 86,2021,4.0,7200000,15 januari 2022,13,122034,vasastan,59,2,None
2,sankt eriksgatan 109,436,2.0,3150000,14 januari 2022,19,126000,vasastan,25,1,None
3,torsgatan 58,1733,2.0,3435000,14 januari 2022,15,110806,vasastan,31,1,1
4,torsplan 8,3950,3.0,10000000,14 januari 2022,11,153846,vasastan,65,3,9
...,...,...,...,...,...,...,...,...,...,...,...
10471,kammakargatan 70,1169,NaN,2800000,19 mars 2013,13,75676,norrmalm,37,2,3
10472,olofsgatan 18,917,NaN,2360000,28 februari 2013,8,73750,norrmalm,32,1,3
10473,regeringsgatan 70 d,3231,NaN,3600000,17 februari 2013,3,52174,norrmalm,69,3,None
10474,drottninggatan 80,2647,NaN,3250000,22 januari 2013,5,73864,norrmalm,44,2,3


# Yelp API calls

In [2]:
yelp_df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/UNFINISHED_yelp_df_CLEANED.csv")


In [7]:
yelp_api = YelpAPI("KulP_1xAbhj4PcwcltixYR5hz4qMJ2aarTp4uNP_bBED4CsgP1nqY0bZrDxRMMSsZYwqvirOQ1Dy--6v3Y2yS4lBVPmfebDVdXGukr74OZEKRNoivTBiORBJ0v_iYXYx")

#A 'suggested search area' of 500 is used, although, the docs tell us the following: 
#This field is used as a suggestion to the search. The actual search radius may be lower 
#than the suggested radius in dense urban areas, and higher in regions of less business density
#hopefully, it does give some quantifiable value as to the 'closeness' to points of interest a apartment has


def get_POI(adress):
    input_adress = adress
    response = yelp_api.search_query(location=input_adress, radius=500, limit=1)
    return response

def get_yelp_values(df, k):
    try:
        for i in range(k, len(df)):
            response = get_POI(df["adress"][i])
            print("Getting values from adress:", df["adress"][i], "from row: " , i)
            
            lat = ( response['region']['center']['latitude'] )
            long = ( response['region']['center']['longitude'] )
            POI = ( response['total'])
            df.at[i, 'Latitude'] = lat
            df.at[i, 'Longitude'] = long
            df.at[i, 'NearbyPOIs'] = POI
            
        return df
    except Exception as e:
        print(e)
        print("\n","error")
        k = i+1
        
        error_handler(df,k)
        
        
def error_handler(df,k):
    get_yelp_values(df,k)
    return 

get_yelp_values(yelp_df,k)

#DONE UNTIL 9989, begin at k = 9990

yelp_df.to_csv("UNFINISHED_yelp_df_CLEANED.csv",index=False)

Getting values from adress: torsgatan 9 from row:  9990
Getting values from adress: johannesgatan 10 from row:  9991
Getting values from adress: tunnelgatan 1a from row:  9992
Getting values from adress: regeringsgatan 70a from row:  9993
Getting values from adress: drottninghusgränd 7 from row:  9994
Getting values from adress: tegnérgatan 55 a from row:  9995
Getting values from adress: kammakargatan 18 from row:  9996
Getting values from adress: tegnérgatan 49 from row:  9997
Getting values from adress: olof palmes gata 12 from row:  9998
Getting values from adress: torsgatan 9 from row:  9999
Getting values from adress: regeringsgatan 70f from row:  10000
Getting values from adress: drottninghusgränd 5 from row:  10001
Getting values from adress: kammakargatan 43 from row:  10002
Getting values from adress: regeringsgatan 70 a from row:  10003
Getting values from adress: wallingatan 13 from row:  10004
Getting values from adress: drottninggatan 73a from row:  10005
Getting values f

Getting values from adress: drottninggatan 73a from row:  10121
Getting values from adress: klara östra kyrkogata 2a from row:  10122
Getting values from adress: olof palmes gata 16 from row:  10123
Getting values from adress: kammakargatan 33 from row:  10124
Getting values from adress: gamla brogatan 25 from row:  10125
Getting values from adress: apelbergsgatan 60 from row:  10126
Getting values from adress: rosengatan 5 from row:  10127
Getting values from adress: olofsgatan 18 from row:  10128
Getting values from adress: regeringsgatan 85 a from row:  10129
Getting values from adress: drottninggatan 73c from row:  10130
Getting values from adress: olof palmes gata 12 from row:  10131
Getting values from adress: tegnérgatan 9 from row:  10132
Getting values from adress: dalagatan 8 from row:  10133
Getting values from adress: drottninghusgränd 2 from row:  10134
Getting values from adress: tegnérgatan 55 a from row:  10135
Getting values from adress: herkulesgatan 22 from row:  101

Getting values from adress: kungsgatan 51 from row:  10252
Getting values from adress: tegnérgatan 55 from row:  10253
Getting values from adress: kammakargatan 64 from row:  10254
Getting values from adress: tegnérgatan 11a from row:  10255
Getting values from adress: johannes plan 1 from row:  10256
Getting values from adress: regeringsgatan 78a from row:  10257
Getting values from adress: herkulesgatan 22 from row:  10258
Getting values from adress: david bagares gata 12a from row:  10259
Getting values from adress: kammakargatan 21 from row:  10260
Getting values from adress: kungsgatan 51 from row:  10261
Getting values from adress: kammakargatan 50 from row:  10262
Getting values from adress: malmskillnadsgatan 60 from row:  10263
Getting values from adress: kammakargatan 21 from row:  10264
Getting values from adress: regeringsgatan 95 from row:  10265
Getting values from adress: regeringsgatan 70 d from row:  10266
Getting values from adress: regeringsgatan 70 a from row:  1026

Getting values from adress: kammakargatan 50 from row:  10383
Getting values from adress: lilla bantorget 25. 6 tr from row:  10384
Getting values from adress: kammakargatan 70 from row:  10385
Getting values from adress: kammakargatan 70 from row:  10386
Getting values from adress: wallingatan 30a from row:  10387
Getting values from adress: regeringsgatan 76 from row:  10388
Getting values from adress: klara tvärgränd 5 from row:  10389
Getting values from adress: kammakargatan 18 from row:  10390
Getting values from adress: kungsgatan 51 from row:  10391
Getting values from adress: dalagatan 8 from row:  10392
Getting values from adress: tegnérgatan 49 from row:  10393
Getting values from adress: dalagatan 8 from row:  10394
Getting values from adress: regeringsgatan 80 from row:  10395
Getting values from adress: regeringsgatan 70 a from row:  10396
Getting values from adress: regeringsgatan 87 from row:  10397
Getting values from adress: herkulesgatan 22 from row:  10398
Getting v

In [16]:
yelp_df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/hemnet_housing_yelp_csv.csv")
